In [96]:
%matplotlib inline
import pandas as pd
import numpy as np
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
import matplotlib as mpl 
#mpl.use("module://mplcairo.qt") 
from mplcairo import operator_t
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [97]:
def get_bins(df, column, n_bins):
    min_v = df[column].min()
    max_v = df[column].max()
    array = []
    
    for i in range(n_bins+1):
        step = max_v/n_bins
        array.append(int(round(step*i, 0)))
    
    return array

In [98]:
## Get the names and open data
renda_path = ('./data/CENSO/PessoaRenda_PB.csv') 

data = pd.read_csv(renda_path,                  
                sep=';', 
                encoding="utf-8",
                dtype={'Cod_setor': str,
                      'V020': float,
                      'V021': float,
                      'V022': float},
                   na_values = ['X'],
                lineterminator='\n')

In [99]:
## Open SHP
setores = gpd.read_file('./data/shp/jp_shp.shp')
setores = setores.to_crs(epsg=3857)

In [100]:
border = setores
border['new_column'] = 0
border = border.dissolve(by='new_column')
border = border[['geometry']]
border['geometry'] = border['geometry'].buffer(0.0001)

In [101]:
# Select only the data we are interested in
#v001 = Moradores em domicilios particular e domicilios coletivos
renda = data[['Cod_setor', 'V020', 'V021', 'V022']]
renda = renda.rename(columns={'V020': 'num_pessoas',
                             'V021': 'num_pessoas_rendimento',
                             'V022': 'rendimento_nominal'})

setores = setores[['geometry', 'NM_BAIRRO', 'CD_GEOCODI']]

In [102]:
# Set the same inde
renda.set_index('Cod_setor', inplace=True)
setores.set_index('CD_GEOCODI', inplace=True)

In [103]:
setores = setores.join(renda)

In [104]:
bairros = setores.dissolve(by='NM_BAIRRO', aggfunc='sum')

In [105]:
bairros_interesse = ['Torre', 'Tambauzinho', 'Expedicionários', 'Miramar', 'Pedro Gondim', 'Estados']
setores_recorte = setores.loc[setores['NM_BAIRRO'].isin(bairros_interesse)]

In [110]:
def get_renda_per_capita(row):
    try:
        renda = row['rendimento_nominal']/row['num_pessoas']
    except (ValueError, ZeroDivisionError):
        return np.nan
    return renda

def get_renda_por_trabalhador(row):
    try:
        renda = row['rendimento_nominal']/row['num_pessoas_rendimento']
    except (ValueError, ZeroDivisionError):
        return np.nan
    return renda

setores_recorte['renda_per_capita'] = setores_recorte.apply(get_renda_per_capita, axis=1)
setores_recorte['renda_por_trabalhador'] = setores_recorte.apply(get_renda_por_trabalhador, axis=1)
bairros['renda_per_capita'] = bairros.apply(get_renda_per_capita, axis=1)
bairros['renda_por_trabalhador'] = bairros.apply(get_renda_por_trabalhador, axis=1)

<ipython-input-110-7fc0744cf7f9>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  setores_recorte['renda_per_capita'] = setores_recorte.apply(get_renda_per_capita, axis=1)
<ipython-input-110-7fc0744cf7f9>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  setores_recorte['renda_por_trabalhador'] = setores_recorte.apply(get_renda_por_trabalhador, axis=1)


In [111]:
bairros

,geometry,num_pessoas,num_pessoas_rendimento,rendimento_nominal,renda_per_capita,renda_por_trabalhador
NM_BAIRRO,,,,,,
Aeroclube,"POLYGON ((-3878845.787 -792159.924, -3878970.3...",8523.0,5975.0,17493369.0,2052.489616,2927.760502
Altiplano Cabo Branco,"POLYGON ((-3876916.727 -797181.774, -3877032.3...",4480.0,2996.0,7316627.0,1633.175670,2442.131842
Alto do Céu,"POLYGON ((-3880526.984 -791742.159, -3880526.7...",13432.0,7110.0,4081383.0,303.855197,574.034177
Alto do Mateus,"POLYGON ((-3885968.582 -797638.000, -3886002.0...",13720.0,7785.0,5279507.0,384.803717,678.164033
Anatólia,"POLYGON ((-3878566.488 -797931.549, -3878554.7...",1061.0,761.0,1549911.0,1460.802074,2036.676741
...,...,...,...,...,...,...
Trincheiras,"POLYGON ((-3883573.691 -795946.209, -3883573.6...",5891.0,3421.0,2684683.0,455.726192,784.765566
Valentina,"POLYGON ((-3878549.633 -804234.168, -3878556.9...",19597.0,12371.0,11102453.0,566.538399,897.458007
Varadouro,"POLYGON ((-3883870.618 -794980.219, -3883870.6...",3181.0,1979.0,1427824.0,448.860107,721.487620


In [95]:
setores_recorte.to_file('./data/RENDA/setores_recorte.shp')
bairros.to_file('./data/RENDA/bairros.shp')